# Process Checks

In [9]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Tickerplant code

In [26]:
from qpython.qconnection import QConnection as qcon
from qpython.qcollection import QDictionary as qdict
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#set qcon variables
import csv
with open('credentials.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Credentials required are {", ".join(row)}')
            line_count += 1
        else:
            host=row[0]
            un=row[1]
            pswd=row[2]
            print(f'\t host:{row[0]} username: {row[1]} password: {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')      
                  

Credentials required are host, username, password
	 host:localhost username: admin password: admin.
Processed 2 lines.


In [11]:
#run torq summary
torq_sum= ! "$(cd "$(dirname "${BASH_SOURCE[0]}")" && pwd)"/torq.sh summary

#convert array to numpy array
summary=np.asarray(torq_sum)

#split each element of array by | character
sum_list = [i.split('|') for i in summary]
df = pd.DataFrame(sum_list)

# take first row and use as header for df
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

#trim whitespace from headers
cols=[]
for i in df.columns:
    cols.append(str.strip(i))
df.columns=cols

#trim whitespace from all objects within dataframe
data = df.select_dtypes(['object'])
df[data.columns] = df.apply(lambda x: x.str.strip())

# function to extract the port number for each process - takes a string argument
def find_portno(process):
    process_info = df.loc[df['PROCESS'] == process]
    PORT = process_info['PORT'].astype(str).astype(int)
    return PORT

#function to decode bytes to strings
def byte_decode(table,cols):
    table[cols] = table[cols].applymap(lambda x: x.decode('utf-8'))

### Process Summary

Table showing process name, status, PID and Port number.
* Process status indicated by colours green (up) and red (down).
* Killtick, tpreplay1 and compression1 should usually have a down status indicated.

In [12]:
# set colour on down processes to red and up processes to green
def colour_down_red(col):
    color = 'red' if 'down' in col else 'green'
    return 'color: %s' % color
df.style.applymap(colour_down_red, subset=['STATUS'])

,TIME,PROCESS,STATUS,PID,PORT
1,15:58:29,discovery1,up,74174,1701
2,15:58:30,tickerplant1,up,74266,1700
3,15:58:30,rdb1,up,74356,1702
4,15:58:30,hdb1,up,74448,1703
5,15:58:30,hdb2,up,74540,1704
6,15:58:30,wdb1,up,74632,1705
7,15:58:30,sort1,up,74726,1706
8,15:58:30,gateway1,up,74818,1707
9,15:58:30,killtick,down,,None
10,15:58:30,monitor1,up,74910,1709


### Count of tables in Tickerplant 

Table to show the count in each table found in the Tickerplant 
* The counts in each of these tables should be 0, as the Tickerplant should not be storing any data.
* If the counts in any of these tables is not 0, this could indicate a slow subscriber. ???

In [13]:
#table to show tables in TP and the counts of each of them
##counts should all be zero
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    tablecounts = q("enlist tables[]!count each value each tables[]", pandas=True)
tablecounts

,logmsg,quote,quote_iex,trade,trade_iex
0,0,0,0,0,0


### Tickerplant Log file size increasing

Checks if log messages in the log file of the tickerplant is increasing.
   * If log messages are increasing, the tickerplant is receiving data.
   * If log messages are not increasing, the tickerplant may not be recieving data. 


In [14]:
#log files are increasing over time
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    log1=(q("hcount .u.L"))
    time.sleep(2)
    log2=(q("hcount .u.L"))
print ("Log file sizes are increasing: ", log1<log2)

Log file sizes are increasing:  True


### Process handles connected to Tickerplant

Table to show the handles of processes connected to the tickerplant and if there are any slow subscribers.
  *  A process may be a slow subscriber if there is a number value in the output queue.

In [15]:
# shows IPC handles with number of bytes waiting in their output queues and what processes are connected
##shows any slow subscribers 
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    zW=q(".z.W[]", pandas=True)
    hprocesses=q("asc select w,u from .clients.clients", pandas=True)
    byte_decode(hprocesses, ['u'])
# assign columns with new names
keys=pd.DataFrame(zW.keys, columns=['handles'])
values=pd.DataFrame(zW.values, columns=['output queue'])
hprocesses=hprocesses.rename(columns={'u':'processes'})
# apply new names
zW2=keys.join(values)
# join zW2 and hprocesses
zW2=zW2.join(hprocesses.processes)
zW2.set_index('handles', inplace=True)
zW2

,output queue,processes
handles,,
6,NaN,rdb
7,NaN,wdb
8,NaN,feed
9,NaN,chainedtp
10,NaN,metrics
11,NaN,iexfeed
12,NaN,admin


## RDB code

In [18]:
with qcon(host, port=find_portno('rdb1'), username=un, password=pswd,timeout=3.0) as q:
    
    #Check tables in rdb are same as tables in tickerplant 
    tables = q('all 1_tables[] in ((exec w from .servers.SERVERS where proctype=`tickerplant)0)("tables[]")')
    #Check count of tables in rdb - data is being sent from the tickerplant 
    tptordb = q('enlist tables[]!count each value each tables[]', pandas=True)
    #Check rdb can be queried
    rdbtquery = q('-5#select from trade', pandas=True)
    #dbqquery = q('-5#select from quote', pandas=True)
    byte_decode(rdbtquery,['sym'])

    #Check that only data from today is present in the rdb tables
    onedatet = q('select Currentdate:all .z.d=distinct (`date$time) from trade', pandas=True)
    #ondedateq = q('select Currentdate:all .z.d=distinct (`date$time) from quote', pandas=True)

### RDB tables

Checks if the tables in the rdb are the same as the tables in the Tickerplant.

In [19]:
print ("RDB tables are same as Tickerplant tables : ", tables)

RDB tables are same as Tickerplant tables :  True


### RDB table counts
Checks counts of tables in the RDB
* A count of zero, indicates no data has been sent from the Tickerplant to the RDB

### RDB table counts

Checks to see if data is being sent from the Tickerplant to the RDB
* Counts for heartbeat and logmsg should be 0
* If counts for the other tables is 0, the RDB has been sent no data for today

In [20]:
tptordb.set_index(tptordb.columns.tolist())

,,,,,
heartbeat,logmsg,quote,quote_iex,trade,trade_iex
0,0,578075,0,120881,0


### RDB Query
Checks to see if tables in the RDB can be queried 
* First table = trade table
* (Second table = quote table)

In [21]:
rdbtquery

,time,sym,price,size,stop,cond,ex
0,2019-10-24 14:59:34.178525,GOOG,72.23,70,False,G,N
1,2019-10-24 14:59:34.178525,HPQ,36.90,10,False,,O
2,2019-10-24 14:59:34.178525,MSFT,28.64,10,False,Z,N
3,2019-10-24 14:59:34.178525,INTC,50.11,40,True,8,N
4,2019-10-24 14:59:34.178525,GOOG,72.26,0,False,K,N


### RDB Date Checks
Check to see if date in RDB tables is the current date

In [ ]:
def color_true(val):
    color  ='pink' if val==False else 'green'
    return 'background-color: %s' %color
onedatet.style.applymap(color_true)

### HDB Code

In [22]:
with qcon(host, port=find_portno('hdb1'), username=un, password=pswd,timeout=3.0) as q:
    
    #Check hdb table counts excl. eod_summary and eod_summary_iex
    hdbtablecount=q('enlist tables[]!count each value each tables[]',pandas=True)

    
    lastdayquery=q('5#select from trade where date=.z.d-1', pandas=True)
    byte_decode(lastdayquery,['sym'])
    

### HDB Query
HDB Table counts 

In [23]:
hdbtablecount.set_index(hdbtablecount.columns.tolist())

,,,,,
heartbeat,logmsg,quote,quote_iex,trade,trade_iex
0,0,7066477,0,1604460,0


### HDB Query
Check to see if HDB tables can be queried

In [24]:
lastdayquery

,date,time,sym,price,size,stop,cond,ex
0,2019-10-23,2019-10-23 14:36:57.564500,MSFT,31.36,69,False,Z,N
1,2019-10-23,2019-10-23 14:36:57.564500,MSFT,31.36,45,False,E,N
2,2019-10-23,2019-10-23 14:36:58.968195,MSFT,31.28,26,False,W,N
3,2019-10-23,2019-10-23 14:37:00.364220,MSFT,31.38,49,False,9,N
4,2019-10-23,2019-10-23 14:37:00.765186,MSFT,31.37,20,False,Z,N
